# Interactively plotting 900GB of 1/32° precipitation data from X-SHiELD

This notebook leverages a number of Python libraries to interactively plot 3-hourly precipitation data from a 1.25 year X-SHiELD simulation.  The data has been regridded to a regular 1/32° latitude-longitude grid and is hosted—free of egress charges—on Google Cloud through the [NOAA Open Data Dissemination (NODD) Program](https://www.noaa.gov/information-technology/open-data-dissemination).

For interactively plotting high resolution data, on the front end we take advantage of the [`geoviews`](https://geoviews.org) and [`datashader`](https://datashader.org/index.html) libraries.  `geoviews` provides the ability to interactively plot gridded Earth-system data, and `datashader` provides the ability to dynamically plot the data at a resolution that can be reasonably quickly rendeded, coarsening data when zoomed out, and plotting it at native resolution when zoomed in.  On the backend we take advantage of the [`kerchunk`](https://fsspec.github.io/kerchunk/) library, coupled with [`xarray`](https://docs.xarray.dev/en/stable/) and [`dask`](https://www.dask.org) to load and interact with the raw netCDF files on the cloud.  `kerchunk` allows us to view the large collection of netCDF files as if it were a chunked zarr store, which we can interact with lazily; this is important since the full size of the dataset is about 900GB.  

`kerchunk` requires some pre-processing to generate the `combined-kerchunk.json` file included in this repository.  This preprocessing is illustrated in the included notebook called `kerchunk-preprocessing.ipynb`.  There is no need to run this notebook, since it only needs to be run once to generate `combined-kerchunk.json`, but it is included for reference; it takes about five minutes total to run, as it needs to read the metadata from O(100) netCDF files in the cloud.

In [1]:
import dask.bag
import dask.diagnostics
import holoviews.operation.datashader as hd
import hvplot.xarray
import geoviews as gv
import numpy as np
import xarray as xr

from cartopy import crs as ccrs
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

In [2]:
SECONDS_PER_THREE_HOURS = 10800
LON_EXTENT = 360
COMBINED_TARGET = "combined-kerchunk.json"


def shift_dataset(ds):
    half_nx = ds.sizes["grid_xt"] // 2
    positive_grid_xt = ds.grid_xt.isel(grid_xt=slice(None, half_nx))
    negative_grid_xt = ds.grid_xt.isel(grid_xt=slice(half_nx, None)) - LON_EXTENT
    shifted_grid_xt = xr.concat([positive_grid_xt, negative_grid_xt], dim="grid_xt")
    ds = ds.assign_coords(grid_xt=shifted_grid_xt)
    return ds.sortby("grid_xt")

In [3]:
ds = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": COMBINED_TARGET,
            "remote_protocol": "gs",
            "remote_options": {"anon": True}
        },
        "consolidated": False
    },
    chunks={}
)

## Convert the longitude coordinate to run from -180° to 180° and the precipitation to units of mm/3hr

Converting from -180° to 180° is required for `geoviews` to handle the data properly; converting to mm/3hr is just to convert the precipitation rate to more intuitive units.

In [4]:
shifted = shift_dataset(ds)
shifted["pr"] = (SECONDS_PER_THREE_HOURS * shifted.pr).astype(np.float32)

In [5]:
shifted

<xarray.Dataset>
Dimensions:       (time: 3640, grid_xt: 11520, bnds: 2, grid_yt: 5760)
Coordinates:
  * grid_yt       (grid_yt) float32 -89.98 -89.95 -89.92 ... 89.92 89.95 89.98
  * time          (time) object 2019-10-20 03:00:00 ... 2021-01-17 00:00:00
  * grid_xt       (grid_xt) float32 -180.0 -180.0 -179.9 ... 179.9 180.0 180.0
Dimensions without coordinates: bnds
Data variables:
    grid_xt_bnds  (time, grid_xt, bnds) float32 dask.array<chunksize=(1, 11520, 2), meta=np.ndarray>
    grid_yt_bnds  (time, grid_yt, bnds) float32 dask.array<chunksize=(1, 5760, 2), meta=np.ndarray>
    pr            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes:
    code_version:  $Name: bronx-10_performance_z1l $
    grid_tile:     N/A
    grid_type:     regular
    history:       fregrid --format netcdf4_classic --input_file /archive/kyc...
    title:         PIRE_production_202103.20191020.00Z.C3072.L79x2_pire

## Make an interactive plot of the precipitation in units of mm/3hr

Note the logarithmic scale of the colorbar to make a greater range of the precipitation rate visible.

In [6]:
gv_ds = gv.Dataset(shifted, ["grid_xt", "grid_yt", "time"], "pr", crs=ccrs.PlateCarree())
images = gv_ds.to(gv.Image)
regridded = hd.regrid(images)
regridded.opts(
    height=500,
    width=1000,
    colorbar=True,
    projection=ccrs.Robinson(),
    logz=True,
    cmap="Blues",
    clim=(0.025, 2500)
) * gv.feature.coastline

:DynamicMap   [time]
   :Overlay
      .Image.I     :Image   [grid_xt,grid_yt]   (pr)
      .Coastline.I :Feature   [Longitude,Latitude]